In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
import pandas as pd
import plotly.express as px

In [2]:
# v3.0
model_name_or_path = "./model/bert-fa-zwnj-base/"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


# model = TFAutoModel.from_pretrained(model_name_or_path)

# text = "ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
# tokenizer.tokenize(text)

In [3]:
df = pd.read_csv("./all_full_cleaned_comment.csv", index_col=0)
labels = ["rejected", "published"]
df.head(3)

,comment,status
0,تا نباشد چوبتر فرمان نبرد گاو وخر این ضرب المث...,published
1,دولتی شش کلاسی همینه,published
2,اگر بی حجابی علت گرانی بود پس چرا در زمان حکوم...,rejected


In [4]:
# results = set()
occurenece = df["comment"].str.lower().str.split(expand=True).stack().value_counts()
# print(results)

In [5]:
occurence_df = pd.DataFrame(occurenece, columns=["count"])
occurence_df["word"] = occurence_df.index
occurence_df = occurence_df.reset_index(drop=True)
occurence_df

,count,word
0,1017170,و
1,705666,به
2,682175,که
3,524646,از
4,513902,این
...,...,...
772217,1,کنیدوتحریمهالغوبشه
772218,1,شمامذاکره
772219,1,نتوتین
772220,1,نعوبق


In [6]:
occurence_df["count"].mean()

44.526492381724424

In [9]:
len(occurence_df.loc[occurence_df["count"] >= 10])

64704

In [34]:
sliced_occurence = occurence_df.loc[(occurence_df["count"] >= 3) & (occurence_df["count"] <= 7)]
fig = px.bar(sliced_occurence, x="word", y="count")
fig.write_html("sliced_comment_words_dist.html")

In [28]:
fig = px.bar(occurence_df, x="word", y="count")
fig.write_html("comment_words_dist.html")

In [36]:
count_words = pd.DataFrame(occurence_df.groupby(by="count").count())
count_words["count"] = count_words.index
count_words = count_words.rename(columns={"word": "word_count","count":"occurenece"})
count_words = count_words.reset_index(drop=True)
fig = px.scatter(count_words, x="occurenece", y="word_count",)
fig.show()

In [188]:
occurence_df.to_excel("occurenece.xlsx")

In [10]:
new_tokenizer = tokenizer.train_new_from_iterator(df["comment"].to_list(), vocab_size=65000, )

In [11]:
new_vocab = new_tokenizer.get_vocab().keys()
old_vocab = tokenizer.get_vocab().keys()


In [12]:
new_added = [x for x in new_vocab if x not in old_vocab]

In [13]:
tokenizer.add_tokens(new_added)

41253

In [14]:
tokenizer_model = input("Enter Tokenizer model name: ")
tokenizer.save_pretrained(f"./model/{tokenizer_model}/")

('./model/aasaam-comment-full-data-bert/tokenizer_config.json',
 './model/aasaam-comment-full-data-bert/special_tokens_map.json',
 './model/aasaam-comment-full-data-bert/vocab.txt',
 './model/aasaam-comment-full-data-bert/added_tokens.json',
 './model/aasaam-comment-full-data-bert/tokenizer.json')